# Clipper Tutorial : R model deployment using RPy2

This tutorial will walk you through the process of starting Clipper, deploying R models to Clipper and creating & querying a Clipper application.

## Starting Clipper

Remember, Docker and Docker-Compose must be installed before deploying Clipper
Make sure your Docker daemon, local or remote, is up and running.

If you'd like to deploy Clipper locally, you can leave the user and key variables blank and set host="localhost". Otherwise, you can deploy Clipper remotely to a machine that you have SSH access to. Set the user variable to your SSH username, the key variable to the path to your SSH key, and the host variable to the remote hostname or IP address.

In [ ]:
# clipper_manager must be on your path:
import sys
import os
sys.path.append(os.path.abspath('../../clipper_admin/'))
import clipper_manager as cm
# Change the username if necessary
user = ""
# Set the path to the SSH key
key = ""
# Set the SSH host
host = ""
clipper = cm.Clipper(host, user, key)

clipper.start()

Congratulations! You now have a running Clipper instance that you can start to interact with.
Now we shall look to forge an R model. 

## Developing an R model  

In [ ]:
import R_model_support as RS

RS.ro.r('data(mtcars)')
model_cars =RS.ro.r('model_R <- lm(mpg~wt,data=mtcars)')

The variable 'model_cars' is linear regression model and the dataset used to train it is "mtcars" provided by R itself. We now have to deploy model_cars in an R model container.

## Deploying R model

Before deploying R model , make sure u have an image for R-Python container. To build the image, run the shell command: "cd (clipper-root)/containers/R-Python/ && docker build -t clipper/r_python_container ."


Now run the following :

In [ ]:
model_name = "the_R_model"


model_added = clipper.deploy_R_model(
    model_name,
    1,
    model_cars,
    "clipper/r_python_container:latest",
    ["time-series", "R"],
    "strings",
    num_containers=1
)
print("Model deploy successful? {success}".format(success=model_added))

Next step is to create an application to carry out predictions with the model.

## Registering Application 

In [ ]:
app_name = "R_app"
app_model=model_name
default_output = "0"

clipper.register_application(
    app_name,
    app_model,
    "strings",
    default_output,
    slo_micros=200000)

Now that 'R_app' is created, let us use it to carry out predictions.  

## Start Serving

Here you should create a 'pandas DataFrame' that will be converted to R dataframe before passing it to model's predict function. The pandas DataFrame should be created in accordance with the deployed R model. 

In [ ]:
df=RS.DataFrame({'wt':[5.43,6.00,7.89],'ht' :[4.32,5.76,7.90]})
RS.start_prediction(df,app_name,host)

You can also convert R data frames to pandas DataFrames and carry out predictions with them.

In [ ]:
df = RS.com.load_data('mtcars')
RS.start_prediction(df,app_name,host)